# 부스팅(Boosting)
- 기본적으로는 랜덤 포래스트와 유사한다.
- 다른 점은 각 트리들이 던지는 답의 오차들을 보정할 수 있도록 오차 함수가 존재한다.
- 이를 통해 성능을 끌어 올린다.
- 너어어어어어어어어어어어어어어무 오래걸려요.ㅠㅠ(차라리 딥러닝을 쓰고 말지.ㅠㅠㅠㅠ)

### 라이브러리 설치
- pip install xgboost
- pip install lightgbm


In [1]:
# 기본

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 분류

In [4]:
# 데이터 준비
train_df = pd.read_parquet('/content/drive/MyDrive/pca_result20.parquet')
test_df = pd.read_parquet('/content/drive/MyDrive/test_pca.parquet')
target_df = pd.read_csv('/content/drive/MyDrive/Segment.csv')

display(train_df.columns)
display(test_df.columns)

Index(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10',
       'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19',
       'PC20'],
      dtype='object')

Index(['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11',
       'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20'],
      dtype='object')

In [5]:
test_df.columns = train_df.columns

In [6]:
display(train_df.columns)
display(test_df.columns)

Index(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10',
       'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19',
       'PC20'],
      dtype='object')

Index(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10',
       'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19',
       'PC20'],
      dtype='object')

In [7]:
target_df

,기준년월,ID,Segment
0,201807,TRAIN_000000,D
1,201807,TRAIN_000001,E
2,201807,TRAIN_000002,C
3,201807,TRAIN_000003,D
4,201807,TRAIN_000004,E
...,...,...,...
2399995,201812,TRAIN_399995,E
2399996,201812,TRAIN_399996,D
2399997,201812,TRAIN_399997,C
2399998,201812,TRAIN_399998,E


In [8]:
encoder1 = LabelEncoder()
Seg = encoder1.fit_transform(target_df['Segment'])


In [9]:
Seg

array([3, 4, 2, ..., 2, 4, 4])

In [11]:
# 입력과 결과로 나눈다.
X = train_df
y = Seg

In [12]:
train_X = X
train_y = y

In [13]:
test_X = test_df

### 기본 모델 사용하기

In [14]:
print(pd.Series(train_y).unique())

[3 4 2 0 1]


In [15]:
print(pd.Series(train_y).value_counts())

4    1922052
3     349242
2     127590
0        972
1        144
Name: count, dtype: int64


In [22]:
model5 = LGBMClassifier(device='gpu', verbose=-1)

kfold = KFold(n_splits=5, shuffle=True, random_state=1)
r1 = cross_val_score(model5, train_X, train_y, scoring='f1_micro', cv=kfold)
print(f'평균 f1 Score : {r1.mean()}')

평균 f1 Score : 0.8334729166666668


In [23]:
model6 = XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor',
                       n_jobs=-1, verbosity=0, use_label_encoder=False)

# 교차 검증
kfold = KFold(n_splits=10, shuffle=True, random_state=1)

r2 = cross_val_score(model6, train_X, train_y, scoring='f1_micro', cv=kfold)
print(f'평균 f1 Score : {r2.mean()}')

평균 f1 Score : 0.8735220833333333


| 클래스 | 샘플 수    | 기본 가중치(역수) | 완화된 가중치(로그) | 추천 가중치 (예시) |
|--------|------------|-------------------|---------------------|--------------------|
| 4      | 1,922,052  | 0.25              | 0.22                | 1                  |
| 3      | 349,242    | 1.37              | 0.95                | 1.5                |
| 2      | 127,590    | 3.76              | 1.5                 | 4                  |
| 0      | 972        | 493.9             | 6.2                 | 50                 |
| 1      | 144        | 3330.5            | 7.8                 | 50                 |


In [29]:
### 가중치 줘보기
class_weights = {0: 40., 1: 155, 2: 4, 3: 1.5, 4: 1}

In [30]:
model3 = LGBMClassifier(device='gpu', class_weight=class_weights, verbose=-1)

kfold = KFold(n_splits=5, shuffle=True, random_state=1)
r3 = cross_val_score(model3, train_X, train_y, scoring='f1_weighted', cv=kfold)
print(f'평균 f1 Score : {r3.mean()}')

평균 f1 Score : 0.8604913725753937


In [32]:
model4 = XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor',
                       n_jobs=-1, verbosity=0, use_label_encoder=False)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

# 클래스별 가중치 예시

f1_scores = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_X, train_y)):
    X_tr, X_val = train_X.iloc[train_idx], train_X.iloc[val_idx]
    y_tr, y_val = train_y[train_idx], train_y[val_idx]   # .iloc → []

    sample_weights = np.array([class_weights[label] for label in y_tr])

    model4.fit(X_tr, y_tr, sample_weight=sample_weights)
    preds = model4.predict(X_val)

    f1 = f1_score(y_val, preds, average='weighted')
    f1_scores.append(f1)
    print(f'Fold {fold+1} F1 Score: {f1:.4f}')


Fold 1 F1 Score: 0.8645
Fold 2 F1 Score: 0.8638
Fold 3 F1 Score: 0.8640
Fold 4 F1 Score: 0.8631
Fold 5 F1 Score: 0.8647
Fold 6 F1 Score: 0.8636
Fold 7 F1 Score: 0.8628
Fold 8 F1 Score: 0.8644
Fold 9 F1 Score: 0.8631
Fold 10 F1 Score: 0.8631


### 모델 하이퍼 파라미터 튜닝

In [ ]:
10/0

In [27]:
# 튜닝할 하이퍼 파라미터 후보 값
# n_estimators : 트리의 개수
# learning_rate : 학습률. 오차 보정을 위해 상수 값들을 수정하는 정도
params = {
    'n_estimators' : [50, 100, 150],
    'learning_rate' : [0.001, 0.01, 0.05, 0.1, 0.3]
}
# 사용할 모델 객체를 생성한다.
model1 = LGBMClassifier(device='gpu', verbose=-1)
# 최적의 하이퍼 파라미터를 찾는다.
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
grid_clf1 = GridSearchCV(model1, param_grid=params, scoring='f1_micro', cv=kfold)
grid_clf1.fit(train_X, train_y)
print(f'최적의 하이퍼 파라미터 : {grid_clf1.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf1.best_score_}')

최적의 하이퍼 파라미터 : {'learning_rate': 0.05, 'n_estimators': 150}
최적의 모델 평균 성능 : 0.8681175


In [25]:
# 튜닝할 하이퍼 파라미터 후보 값
# booster : 내부에서 사용할 알고리즘. gbtree - 결정트리, gblinear - 선형모델
# n_estimators : 트리의 개수
# learning_rate : 학습률. 오차 보정을 위해 상수 값들을 수정하는 정도

params = {
    'booster' : ['gbtree'],  # 'gblinear' 제거
    'n_estimators' : [50, 100, 150],
    'learning_rate' : [0.001, 0.01, 0.05, 0.1, 0.3]
}

model2 = XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor',
                       n_jobs=-1, verbosity=0, use_label_encoder=False)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

grid_clf2 = GridSearchCV(model2, param_grid=params, scoring='f1_micro', cv=kfold, n_jobs=-1)
grid_clf2.fit(train_X, train_y)

print(f'최적의 하이퍼 파라미터 : {grid_clf2.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf2.best_score_:.4f}')

최적의 하이퍼 파라미터 : {'booster': 'gbtree', 'learning_rate': 0.3, 'n_estimators': 150}
최적의 모델 평균 성능 : 0.8746


### 예측을 수행한다

In [34]:
# 예측할 데이터를 읽어온다.
df1 = pd.read_parquet('/content/drive/MyDrive/test_pca.parquet')
df2 = df1.copy()
df3 = df2.copy()
df4 = df3.copy()
df5 = df4.copy()
df6 = df5.copy()

df1.columns = train_X.columns
df2.columns = train_X.columns
df3.columns = train_X.columns
df4.columns = train_X.columns
df5.columns = train_X.columns
df6.columns = train_X.columns
df1.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20
0,5449.770399,-15457.518923,7975.048624,-2971.418082,-1137.004720,4011.962214,11551.144045,3357.707219,-2924.619961,-1328.630829,6456.306948,5052.190399,-1758.037008,757.974686,-3801.911975,-3675.470214,89.699894,192.708209,-2603.045347,6486.179120
1,-39956.605716,26232.031295,1163.543135,-8719.802068,-6359.839526,-9991.101935,-4302.131549,15137.268688,10291.412329,815.171496,1105.501987,2826.930744,4697.819084,3187.565765,-1314.235267,-2610.468020,1837.496242,2076.203385,2799.656647,-765.383592
2,91343.368168,20400.129647,49117.164561,18297.225673,-3701.289750,445.182607,-8089.748593,9430.604063,-8090.458211,-63.631244,-2532.892084,6031.427744,-2219.302720,-1307.780108,35.376295,1316.920654,569.088409,-2300.245020,-2180.985726,1976.296339
3,-60622.721453,-2779.211142,172.147101,-1520.579240,494.335640,-2245.292728,-2498.491927,4213.196652,2446.223055,-925.403803,2079.755754,2406.735903,-1541.601982,2355.289054,-669.735345,253.566145,-410.776255,611.909119,677.146892,1175.953003
4,-48203.789469,188.499891,8676.875441,-8345.019485,-4152.234122,-8421.964890,-2943.931765,4537.336623,1748.248608,-1903.155119,2405.428335,4011.805170,436.572392,3449.701387,-1297.313124,-1117.081428,114.225211,538.235245,1567.426358,490.377240


In [37]:
model1

LGBMClassifier(device='gpu', verbose=-1)

In [38]:
model2

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=-1,
              num_parallel_tree=None, predictor='gpu_predictor', ...)

In [39]:
model3

LGBMClassifier(class_weight={0: 40.0, 1: 155, 2: 4, 3: 1.5, 4: 1}, device='gpu',
               verbose=-1)

In [44]:
model3.fit(train_X, train_y)

LGBMClassifier(class_weight={0: 40.0, 1: 155, 2: 4, 3: 1.5, 4: 1}, device='gpu',
               verbose=-1)

In [40]:
model4

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=-1,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [35]:
# 하이퍼파라미터모델
grid_clf1

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             estimator=LGBMClassifier(device='gpu', verbose=-1),
             param_grid={'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.3],
                         'n_estimators': [50, 100, 150]},
             scoring='f1_micro')

In [36]:
# 하이퍼파라미터모델
grid_clf2

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=-1, num_parallel_tree=None,
                                     predictor='gpu_predictor', ...),
             n_jobs=-1,
             param_grid={'booster': ['gbtree'],
                         'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.3],
                         'n_estimators': [50, 100, 150]},
             scoring='f1_micro')

In [41]:
# 예측한다
y_pred1 = grid_clf1.predict(df1)

In [42]:
y_pred2 = grid_clf1.predict(df2)

In [45]:
y_pred3= model3.predict(df3)

In [46]:
y_pred4= model4.predict(df4)

In [48]:
model5.fit(train_X, train_y)
y_pred5= model5.predict(df5)

In [49]:
model6.fit(train_X, train_y)
y_pred6= model6.predict(df6)

In [50]:
# 결과 데이터를 복원한다.
result_data1 = encoder1.inverse_transform(y_pred1)
result_data2 = encoder1.inverse_transform(y_pred2)
result_data3 = encoder1.inverse_transform(y_pred3)
result_data4 = encoder1.inverse_transform(y_pred4)
result_data5 = encoder1.inverse_transform(y_pred5)
result_data6 = encoder1.inverse_transform(y_pred6)

In [57]:
# 예측 결과데이터를 만들기 위해 ID와 기준년월이 있는 테스트파일을 불러온다.
submission_df = pd.read_parquet('/content/drive/MyDrive/신용카드데이터_test.parquet')
submission_df1 = submission_df.copy()
submission_df2 = submission_df.copy()
submission_df3 = submission_df.copy()
submission_df4 = submission_df.copy()
submission_df5 = submission_df.copy()
submission_df6 = submission_df.copy()

In [58]:
# 원하는 컬럼만 불러온다
submission_df1 = submission_df1[['ID','기준년월']]
submission_df2 = submission_df2[['ID','기준년월']]
submission_df3 = submission_df3[['ID','기준년월']]
submission_df4 = submission_df4[['ID','기준년월']]
submission_df5 = submission_df5[['ID','기준년월']]
submission_df6 = submission_df6[['ID','기준년월']]
# 예측 결과를 저장한다.
submission_df1['Segment'] = result_data1
submission_df2['Segment'] = result_data2
submission_df3['Segment'] = result_data3
submission_df4['Segment'] = result_data4
submission_df5['Segment'] = result_data5
submission_df6['Segment'] = result_data6

print('저장완료')

저장완료


In [78]:
submission_df6['Segment'].value_counts()


,count
Segment,
E,500685
D,74243
C,25036
A,36


In [79]:
# 일단 submission_df1~6까지 모두 csv파일로 저장해둔다.
submission_df1.to_csv('/content/drive/MyDrive/data/LGBM하이퍼파라미터20.csv', index=False)
submission_df2.to_csv('/content/drive/MyDrive/data/XGB하이퍼파라미터20.csv', index=False)
submission_df3.to_csv('/content/drive/MyDrive/data/LGBM가중치20.csv', index=False)
submission_df4.to_csv('/content/drive/MyDrive/data/XGB가중치20.csv', index=False)
submission_df5.to_csv('/content/drive/MyDrive/data/LGBM20.csv', index=False)
submission_df6.to_csv('/content/drive/MyDrive/data/XGB20.csv', index=False)

print('모든 submission 데이터프레임이 CSV 파일로 저장되었습니다.')


모든 submission 데이터프레임이 CSV 파일로 저장되었습니다.


In [80]:
submission_df6

,ID,기준년월,Segment
0,TEST_00000,201807,E
1,TEST_00001,201807,E
2,TEST_00002,201807,D
3,TEST_00003,201807,E
4,TEST_00004,201807,E
...,...,...,...
599995,TEST_99995,201812,E
599996,TEST_99996,201812,E
599997,TEST_99997,201812,E
599998,TEST_99998,201812,C


f1스코어를 보았을 때
model6인 하이퍼파라미터 xgboost가 가장 높으므로 model2를 선택한다.

In [68]:
# 예측 결과데이터의 예시를 보면 10만개의 데이터행으로 이루어져있으므로
# 12월 데이터만 만들어준다.
submission_df2_12 = submission_df2[submission_df6['기준년월'] == 201812]

In [70]:
submission_df2_12['Segment'].value_counts()

,count
Segment,
E,82772
D,13038
C,4106
A,43
B,41


In [71]:
submission_df2_12.drop('기준년월', axis=1, inplace=True)
# 제대로 나누어졌는지 확인해본다.
submission_df2_12

,ID,Segment
500000,TEST_00000,E
500001,TEST_00001,E
500002,TEST_00002,E
500003,TEST_00003,E
500004,TEST_00004,E
...,...,...
599995,TEST_99995,E
599996,TEST_99996,E
599997,TEST_99997,E
599998,TEST_99998,C


In [73]:
# 12월 데이터만 제출하기로 정해졌으므로 12월 데이터만 내보낸다.
submission_df2_12.to_csv('/content/drive/MyDrive/data/Kaggle_Hype_12.csv', index=False)